# Tweet Scraper

Este es el notebook que usaremos para el scrapeo de datos


## Librerias

Para el scrapeo usaremos las siguientes librerias:
* Sntwitter
* Pandas

In [ ]:
import datetime
from datetime import timedelta

import snscrape.modules.twitter as snstwitter

import pandas as pd
import seaborn as sns

from tqdm import tqdm

## Parametros



In [ ]:
columnNames = [
    'Datetime', 'Tweet Id', 'Text', 
    'NumReplies', 'NumRetweets', 'NumLikes', 
    'IDOriginalRetweeted', 'Username', 'isVerified'
]

condition_query = '"BTC" OR "bitcoin" since:{since} until:{until} lang:{lang}'

## Función obtener tweets


In [ ]:
def get_tweets(date_from, date_until, tweet_limit = -1, lang="en"):
    """
        Función para scrapear tweets entre fechas
        
        Parameters:
        date_from (datetime.date): Fecha de comienzo del scrapping
        date_until (datetime.date): Fecha hasta la que se realiza el scrapping. Fecha no incluida.
        tweet_limit (int): Limite de tweets al dia. -1 si no se quiere limite
        
        Returns:
        Lista de Valores del tweet
    """
    tweet_list = []
    while(date_from != date_until):
        print("Day " + str(date_from))
        a = datetime.datetime.now()

        format_string = condition_query.format(
            since=str(date_from),
            until=str(date_from + timedelta(days=1)),
            lang=lang
        )
        
        for i, tweet in enumerate(snstwitter.TwitterSearchScraper(format_string).get_items()):
            if i >= tweet_limit and tweet_limit != -1:
                break
                
            if(i%2500==0):
                print(i , " / " , tweet_limit)
            
            tweet_list.append(
                [
                    tweet.date, tweet.id, tweet.content,
                    tweet.replyCount, tweet.retweetCount,
                    tweet.likeCount, tweet.retweetedTweet,
                    tweet.user.username, tweet.user.verified
                ]
            )
        
        date_from += timedelta(days=1)
        b = datetime.datetime.now()
        print(b-a)

    return tweet_list

## Ejemplo de uso

In [ ]:
date_from = datetime.date(2018, 1, 1)
date_until = datetime.date(2018, 2, 1)
max_tweet = 1500

directory = 'data/tweets/' + str(date_from) + '/' +  str(date_until)
file_name = directory + '/tweet_list_' + str(max_tweet) +'.csv'

In [ ]:
tweet_list = get_tweets(date_from, date_until, tweet_limit = max_tweet)

tweet_df = pd.DataFrame(tweet_list, columns=columnNames)
del tweet_list

### Escritura de dataset

In [ ]:
from pathlib import Path

Path(directory).mkdir(parents=True, exist_ok=True)
tweet_df.to_csv(file_name, sep=',', index=False)

### Lectura de ficheros ya existentes

In [ ]:
tweet_df = pd.read_csv(file_name, sep=',')

tweet_df["Datetime"] = pd.to_datetime(tweet_df["Datetime"])

## Creación de nuevas columnas del DataFrame

In [ ]:
tweet_df['sentiment'] = ""
tweet_df['round_time'] = ""

tweet_df["sentiment"] = pd.to_numeric(tweet_df["sentiment"])
tweet_df["round_time"] = pd.to_datetime(tweet_df["round_time"])

## Analisis de sentimientos

In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

### Preparación del Analisis

En caso de ser nuestra primera ejecución, deberemos instalar un conjunto de datasets utiles para *nltk*.

In [ ]:
nltk.download([     
    "names",
    "stopwords",
    "state_union",
    "twitter_samples",
    "movie_reviews",
    "averaged_perceptron_tagger",
    "vader_lexicon",
    "punkt",
])

### Limpieza de dataset

In [ ]:
import re
import emoji

def clean_tweet(tweet):
    tweet = re.sub("@[A-Za-z0-9_]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = emoji.get_emoji_regexp().sub(r'', tweet)
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    return tweet

In [99]:

def get_round_time(data):
    return data.replace(second=0, microsecond=0, minute=0)

def get_sentiment(text):
    """
        Cleans and analyzes text.
        
        Parameters:
        text(string): Bunch of text
        
        Returns:
        Compound sentiment from NLTK polarity
    
    """
    
    clean_text = clean_tweet(text)
    polarity = sia.polarity_scores(clean_text)
    return polarity['compound']

def prepare_data(tweet_df, verbose=False):
    tweet_df['round_time'] = tweet_df['Datetime'].round('h')
    
    for index, row in tweet_df.iterrows():
        tweet_df.loc[i, 'sentiment'] = get_sentiment(tweet_df['Text'].iloc[i])
            

In [101]:
import time
start = time.time()
prepare_data(tweet_df)
end = time.time()
print(end-start)

27.763259172439575


#### Timinng de las funciones

iterrows . 20
.map       27


## Analisis de Sentimiento

Eliminaremos los valores nulos ya que parece que cuando el algoritmo no es capaz de determinar el sentimiento, tiende a ponerle un 0, creando una desviación del sentimiento real.

Lo primero que haremos será mostrar la **media del sentimiento** y una **gráfica de distribución del sentimiento**.

In [ ]:
average_sentiment = tweet_df[tweet_df['sentiment'] != 0]['sentiment'].mean()

print(f"Average sentiment: {average_sentiment}")

sns.distplot(tweet_df[tweet_df['sentiment'] != 0]['sentiment'])